In [ ]:
import scanpy as sc

# Only needed for processing
import numpy as np
import pandas as pd
import os 
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 500)
import liana as li


sc.set_figure_params(dpi=300,dpi_save=300, format="svg",fontsize=10)
sns.set_style('white', rc={
    'xtick.bottom': True,
    'ytick.left': True,
})
plt.rcParams["font.family"] = "Arial"
output_fol = '/home/carsten/alvira_bioinformatics/uterus/data/pilot/240925_atlas_figure_pilot'


In [ ]:
adata = sc.read('/home/carsten/alvira_bioinformatics/uterus/data/single_cell_files/scanpy_files/uterus_processed_celltyped.gz.h5ad')
ct_order=['Ciliated', 'Epithelial', 'Glandular', 'Lumenal',
         'Capillary','Lymphatic EC', 'Macrovascular',
          'Basophil', 'Dendritic', 'Myeloid','Proliferative myeloid','NK cell',  'T cell',
          'Matrix fibroblast','Vessel fibroblast','Trophoblast','Uterine smooth muscle', 'Vascular smooth muscle',
         ]
adata.obs['Cell Subtype'] = pd.Categorical(adata.obs['Cell Subtype'], categories=ct_order)
adata.obs['celltype_endometrial'] = [y if x !='Endometrial' else x for x,y in zip(adata.obs['Lineage'], adata.obs['Cell Subtype'])]

In [ ]:
### FIGURE 2 
output = f'{output_fol}/figure2'
os.makedirs(output, exist_ok=True)
sc.settings.figdir = output


In [ ]:
with plt.rc_context({"figure.figsize": (3, 3), "figure.dpi": (300)}):
    sc.pl.umap(adata,color='celltype_endometrial',groups=['Trophoblast','Endometrial'],title='',frameon=False,na_in_legend=False,save='endometrial_trophoblast.svg')
    sc.pl.umap(adata,color='Lineage',frameon=False,save='lineage.svg')
    sc.pl.umap(adata, color='Cell Subtype',frameon=False,save='cell_subtype.svg')
    np.random.seed(0)
    random_indices = np.random.permutation(list(range(adata.shape[0])))
    sc.pl.umap(adata[random_indices,:], color='Patient',alpha=0.5,frameon=False,save='patient.svg')

In [ ]:
with plt.rc_context({"figure.figsize": (1.5, 1.5), "figure.dpi": (300)}):
    sc.pl.umap(adata,color=["CDH5", "PAEP", "PTPRC", "COL1A1",],alpha=0.5,ncols=2, cmap='viridis',frameon=False,save='lineage_markers.svg')

In [ ]:
patient_color_dict = {}
for x, y in zip(adata.obs['Patient'].cat.categories, adata.uns['Patient_colors']):
    patient_color_dict[x]=y

In [ ]:
df=pd.DataFrame(sc.get.obs_df(adata,['Cell Subtype','Patient']).groupby('Cell Subtype')['Patient'].value_counts()['Trophoblast']).T
palette= [patient_color_dict[x] for x in df.columns]
fig, ax = plt.subplots(1,1, figsize=(2.5,2))
ax = sns.barplot(df, palette=palette)
ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
ax.set_ylabel('# of Trophoblast cells')
fig.savefig(f'{output}/barplot_trophoblast_patient_number.svg',dpi=300,bbox_inches='tight')


In [ ]:
df=pd.DataFrame(sc.get.obs_df(adata,['Lineage','Patient']).groupby('Lineage')['Patient'].value_counts()['Endometrial']).T
palette= [patient_color_dict[x] for x in df.columns]
fig, ax = plt.subplots(1,1, figsize=(2.5,2))
ax = sns.barplot(df, palette=palette)
ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
ax.set_ylabel('# of Endometrial cells')
fig.savefig(f'{output}/barplot_endometrial_patient_number.svg',dpi=300,bbox_inches='tight')


In [ ]:
gene_dictionary = {'Ciliated':['CFAP43','HYDIN','DNAH6','CDHR3'], 
 'Epithelial':['UST','DST','PDE10A','COL12A1'],
 'Glandular':['PTPRR','RIMS2','ZPLD1','VCAN'],
 'Lumenal':['RMST','FBXL7','DLGAP1','RXFP1'],
 'Capillary':['NOSTRIN','ADGRL2','TLL1','TGFBR3'],
       'Lymphatic EC':['CD36','RELN','MPP7','PROX1'],
 'Macrovascular':['SOX5','EFNB2','GJA5','VEGFC'],
 'Basophil':['NTM','SLC24A3','CPA3','IL18R1'],
 'Dendritic':['CCSER1','WDFY4','AFF3','GNG7'], 
 'Myeloid':['RBM47','MS4A6A','MRC1','TNS3'],
       'NK cell':['NCAM1','GNLY','TXK','KCNQ5'],
 'Proliferative myeloid':['CENPP','NSD2','ATAD2','DIAPH3'], 
 'T cell':['THEMIS','INPP4B','SLFN12L','PRKCQ'], 
 'Matrix fibroblast':['CCDC80','MEG3','TNXB','DNM1'],
       'Trophoblast':['HPGD','GPR78','LVRN','RBM47'], 
 'Uterine smooth muscle':['DPP6','LINGO2','OXTR','KCNMA1'],
 'Vascular smooth muscle':['ACTA2','TAGLN','NOTCH3','ADGRL3','PDE3A','FHL5'],
       'Vessel fibroblast':['RORA','RORB','THSD7A','DTNA'],
 }
gene_ls = ['PAEP','CDH5','PTPRC','COL1A1'] 
for k in ct_order:
    gene_ls+=gene_dictionary[k]

In [ ]:
sc.pl.DotPlot(adata, gene_ls, groupby='Cell Subtype',standard_scale='var').style(cmap='Reds').legend(width=2).savefig(f'{output}/dotplot_all_celltypes.svg',dpi=300)
sc.pl.MatrixPlot(adata, gene_ls, groupby='Cell Subtype',standard_scale='var').style(cmap='plasma').legend(width=2).savefig(f'{output}/matrixplot_all_celltypes.svg',dpi=300)


In [ ]:
celltype_color_dict = {}
for x, y in zip(adata.obs['Cell Subtype'].cat.categories, adata.uns['Cell Subtype_colors']):
    celltype_color_dict[x]=y
# Calculate value counts
value_counts = adata.obs['Cell Subtype'].value_counts()

# Create pie chart
fig, ax = plt.subplots(1,1, figsize=(3,3))
ax = value_counts.plot.pie(y=False,
                           colors=[celltype_color_dict[x] for x in value_counts.index],
                           labeldistance=None,
#                            explode=[0.01 for x in value_counts.index],
                           ax=ax
                          )
labels=[f'{x} ({value_counts[x]})' for x in value_counts.index]
f = lambda m,c: plt.plot([],[],marker=m, color=c, ls="none")[0]
handles = [f("o", celltype_color_dict[x]) for x in value_counts.index]
plt.legend(handles,labels,title='', ncol=2, frameon=False,fancybox=False, shadow=False, bbox_to_anchor=(1.0, 0.8))
ax.set_title('Cell Subtype distibution')
ax.set_ylabel('')
fig.savefig(f'{output}/pieplot_celltype_number.svg',dpi=300,bbox_inches='tight')



In [ ]:
### FIGURE 4
output = f'{output_fol}/figure4'
os.makedirs(output, exist_ok=True)
sc.settings.figdir = output

In [ ]:
adata

In [ ]:
adata_mes =adata.copy()
adata_mes.obsm['X_umap'] = adata_mes.obsm['X_umap_Mesenchymal']
adata_mes=adata_mes[adata_mes.obs['Lineage']=='Mesenchymal']
adata_mes

In [ ]:
with plt.rc_context({"figure.figsize": (3, 3), "figure.dpi": (300)}):
    sc.pl.umap(adata_mes,color='Cell Subtype',title='',frameon=False,save='mesenchymal.svg')
    sc.pl.umap(adata_mes,color='Cell Subtype',title='',frameon=False,groups = ['Uterine smooth muscle','Vascular smooth muscle'],na_in_legend=False,save='mesenchymal_just_smooth_muscle.svg')

In [ ]:
adata_sm = adata_mes[adata_mes.obs['Cell Subtype'].isin(['Uterine smooth muscle','Vascular smooth muscle'])]
adata_sm.obs['Cell Subtype_abv'] = ['USM' if x =='Uterine smooth muscle' else 'VSM' for x in adata_sm.obs['Cell Subtype']]
with plt.rc_context({"figure.figsize": (1.5,1.5), "figure.dpi": (300)}):
    for color in gene_dictionary['Uterine smooth muscle'] + gene_dictionary['Vascular smooth muscle']:
        sc.pl.umap(adata_mes,color=color,alpha=0.5, wspace=0.0,cmap='viridis',frameon=False,save=f'{color}.svg')
        
with plt.rc_context({"figure.figsize": (2.5,1.5), "figure.dpi": (300)}):
    for color in gene_dictionary['Uterine smooth muscle'] + gene_dictionary['Vascular smooth muscle']:
        sc.pl.violin(adata_sm,
             color,
             groupby='Cell Subtype_abv',
            xlabel=' ',
            stripplot=False,
                     palette= [celltype_color_dict['Uterine smooth muscle'],celltype_color_dict['Vascular smooth muscle']],
            inner='box',
            frameon=False,
            save=f'{color}.svg')
        


In [ ]:
sc.pl.DotPlot(adata_mes, 
              ['ACTA2','TAGLN'], 
                 standard_scale='var',
              groupby=['Cell Subtype']).style(cmap='Reds').swap_axes().legend(width=2).savefig(f'{output}/dotplot_smooth_muscle.svg',dpi=300)


In [ ]:
adata.raw = adata
from liana.mt import rank_aggregate
rank_aggregate(adata, groupby='Cell Subtype', expr_prop=0.1, resource_name='consensus', verbose=True, key_added='liana_ccc_table')
adata

In [ ]:
df = adata.uns['liana_ccc_table']
df_vsm = df.loc[((df['source']=='Vascular smooth muscle')|(df['target']=='Vascular smooth muscle'))
               &((df['source'].isin(['Capillary','Macrovascular']))|(df['target'].isin(['Capillary','Macrovascular'])))
               ]
df_vsm.head(50)

In [ ]:
df_usm = df.loc[((df['source']=='Uterine smooth muscle')&(df['target']=='Uterine smooth muscle'))
              
               ]
df_usm

In [ ]:
adata.uns['liana_ccc_table'].sort_values('receptor_complex', inplace=True)
fig = li.pl.dotplot(adata = adata,
             colour='magnitude_rank',
              size='specificity_rank',
              inverse_size=True,
              inverse_colour=True,
              source_labels=['Macrovascular', 'Capillary'],
              target_labels=['Uterine smooth muscle','Vascular smooth muscle'],
              figure_size=(8, 7),
              orderby='receptor_complex',
              orderby_ascending=True,

              filterby='receptor_complex',
              filter_lambda=lambda x: x.startswith('NOTCH'),
              uns_key='liana_ccc_table', 
             )
fig.save(f'{output}/ccc_notch_vsm_usm.png',dpi=300)

In [ ]:
fig = li.pl.dotplot(adata = adata,
             colour='magnitude_rank',
              size='specificity_rank',
              inverse_size=True,
              inverse_colour=True,
              source_labels=['Uterine smooth muscle','Vascular smooth muscle'],
              target_labels=['Uterine smooth muscle','Vascular smooth muscle'],
              figure_size=(8, 7),
              orderby='specificity_rank',
              orderby_ascending=True,
                    top_n=15,
              uns_key='liana_ccc_table', 
             )
fig.save(f'{output}/ccc_vsm_usm.png',dpi=300)

In [ ]:
adata.uns['liana_ccc_table']

In [ ]:
sc.pl.dotplot(adata,['NCAM1','CACNA1C'],groupby='Cell Subtype')

In [ ]:
### FIGURE 5
output = f'{output_fol}/figure5'
os.makedirs(output, exist_ok=True)
sc.settings.figdir = output

In [ ]:
### FIGURE 6
output = f'{output_fol}/figure6'
os.makedirs(output, exist_ok=True)
sc.settings.figdir = output

In [ ]:
adata_usm = adata_mes[(adata_mes.obs['Cell Subtype'].isin(['Uterine smooth muscle']))
                     &(adata_mes.obs['GroupContract'].isin(['TNL-GC','TL-ND']))
                     ].copy()
# sc.pp.scale(adata_usm)
adata_usm

In [ ]:
usm_degs_labor = pd.read_excel('/home/carsten/alvira_bioinformatics/uterus/data/figures/deg/pseudobulk/TNL_GC_v_TL_ND_pseudobulk_comparisons.xlsx',
                              sheet_name='Uterine smooth muscle',
                              index_col=0)
usm_degs_labor = usm_degs_labor.loc[usm_degs_labor['padj']<0.05]
usm_degs_labor = usm_degs_labor.loc[abs(usm_degs_labor['log2FoldChange'])>1]
usm_degs_labor = usm_degs_labor.loc[usm_degs_labor['baseMean']>100]

usm_degs_labor = usm_degs_labor.sort_values('log2FoldChange')
usm_degs_labor

In [ ]:
top_n=10
plot_degs = usm_degs_labor.sort_values('log2FoldChange').index.to_list()[:top_n]+ usm_degs_labor.sort_values('log2FoldChange',ascending=False).index.tolist()[:top_n]
sc.pl.MatrixPlot(adata_usm, 
              plot_degs, 
                 standard_scale='var',
#                  vmin=-0.5,
#                  vmax=0.5,
              groupby=['Group','Patient']).style(cmap='magma').legend(width=2).savefig(f'{output}/matrixplot_uterine_smooth_muscle_labor_degs.svg',dpi=300)


In [ ]:
sc.pl.DotPlot(adata_usm, 
              plot_degs, 
                 standard_scale='var',
#                  vmin=-0.5,
#                  vmax=0.5,
              groupby=['Group','Patient']).style(cmap='Reds').legend(width=2).savefig(f'{output}/dotplot_uterine_smooth_muscle_labor_degs.svg',dpi=300)


In [ ]:
adata.obs['Patient']

In [ ]:
pique_genes = {'up':['PI15','ADAMTS9',
'SLC7A5',
'BOP1',
'CXCL2',
'IL6',
'IER3',
'MTHFD1L',
'DNAJB11'],
              'down':[
                  'ITIH5',
'PGM5',
'SLC2A4',
'CTXN1',
'SSPN',
'CSDC2',
'PCYOX1',
'FABP3',
'TMEM106C',
'PLN',
'KCNAB1',]}

sc.pl.MatrixPlot(adata_usm, 
              pique_genes, 
                 standard_scale='var',
              groupby=['Group','Patient']).style(cmap='magma').legend(width=2).savefig(f'{output}/dotplot_uterine_smooth_muscle_labor_degs.svg',dpi=300)


In [ ]:
### FIGURE 7
output = f'{output_fol}/figure7'
os.makedirs(output, exist_ok=True)
sc.settings.figdir = output

In [ ]:
usm_degs = pd.read_excel('/home/carsten/alvira_bioinformatics/uterus/data/figures/deg/pseudobulk/TNL_GC_v_TNL_BC_pseudobulk_comparisons.xlsx',
                              sheet_name='Uterine smooth muscle',
                              index_col=0)
usm_degs_atony = usm_degs.loc[usm_degs['padj']<0.1]
usm_degs_atony = usm_degs_atony.loc[abs(usm_degs_atony['log2FoldChange'])>0.5]

usm_degs_atony = usm_degs_atony.sort_values('log2FoldChange')
usm_degs_atony

In [ ]:
usm_degs["$-Log_{10}$(padj)"] = usm_degs["padj"].apply(lambda x: -np.log10(x))
usm_degs["$Log_{2}$(FC)"] = usm_degs["log2FoldChange"]
color = []
for x in usm_degs.index:
    tmp = usm_degs.loc[x]
    if tmp["log2FoldChange"] > 0.5 and tmp["padj"] < 0.1:
        color.append("Upregulated")
    elif tmp["log2FoldChange"] < -0.5 and tmp["padj"] < 0.1:
        color.append("Downregulated")
    else:
        color.append("NS")

usm_degs["color"] = color
fig, ax = plt.subplots(1, 1, figsize=(4, 3))

sns.scatterplot(
    data=usm_degs,
    y="$-Log_{10}$(padj)",
    x="$Log_{2}$(FC)",
    hue="color",
    hue_order=["Upregulated", "NS", "Downregulated"],
    palette=["red", "grey", "blue"],
    s=10,
    linewidth=0,
    ax=ax,
)
ax.get_legend().remove()
sns.despine(fig=fig,ax=ax,top=True,right=True)
fig.savefig(f"{output}/usm_atony_volcano.png", dpi=300, bbox_inches="tight")

In [ ]:
adata_usm = adata_mes[(adata_mes.obs['Cell Subtype'].isin(['Uterine smooth muscle']))
                     &(adata_mes.obs['GroupContract'].isin(['TNL-GC','TNL-BC']))
                     ].copy()
top_n=10
plot_degs = usm_degs_atony.sort_values('log2FoldChange').index.to_list()[:top_n]+ usm_degs_atony.sort_values('log2FoldChange',ascending=False).index.tolist()[:top_n]
sc.pl.MatrixPlot(adata_usm, 
              plot_degs, 
                 standard_scale='var',
#                  vmin=-0.5,
#                  vmax=0.5,
              groupby=['Contractility','Patient']).style(cmap='magma').legend(width=2).savefig(f'{output}/matrixplot_uterine_smooth_muscle_labor_degs.svg',dpi=300)


In [ ]:
bulk_degs = pd.read_excel('/home/carsten/alvira_bioinformatics/uterus/data/bulk/Result_X202SC22123874-Z01-F001/4.Differential/1.deglist/REDvsGD/REDvsGD_deg.xlsx',
                         engine='openpyxl')
bulk_degs_atony = bulk_degs.loc[bulk_degs['padj']<0.1]
bulk_degs_atony = bulk_degs_atony.loc[abs(bulk_degs_atony['log2FoldChange'])>0.5]

bulk_degs_atony = bulk_degs_atony.sort_values('log2FoldChange')
bulk_degs_atony

In [ ]:
sc.pl.dotplot(adata,[x for x in bulk_degs_atony['gene_name'].values.tolist() if x in adata.var_names],
                     groupby='Cell Subtype',
             save='bulk_atony_degs')

In [ ]:
bulk_degs["$-Log_{10}$(padj)"] = bulk_degs["padj"].apply(lambda x: -np.log10(x))
bulk_degs["$Log_{2}$(FC)"] = bulk_degs["log2FoldChange"]
color = []
for x in bulk_degs.index:
    tmp = bulk_degs.loc[x]
    if tmp["log2FoldChange"] > 0.5 and tmp["padj"] < 0.1:
        color.append("Upregulated")
    elif tmp["log2FoldChange"] < -0.5 and tmp["padj"] < 0.1:
        color.append("Downregulated")
    else:
        color.append("NS")

bulk_degs["color"] = color
fig, ax = plt.subplots(1, 1, figsize=(4, 3))

sns.scatterplot(
    data=bulk_degs,
    y="$-Log_{10}$(padj)",
    x="$Log_{2}$(FC)",
    hue="color",
    hue_order=["Upregulated", "NS", "Downregulated"],
    palette=["red", "grey", "blue"],
    s=10,
    linewidth=0,
    ax=ax,
)
ax.get_legend().remove()
sns.despine(fig=fig,ax=ax,top=True,right=True)
fig.savefig(f"{output}/bulk_atony_volcano.png", dpi=300, bbox_inches="tight")